In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm

# 資料夾結構
dataset<br>
├─ asparagus<br>
├─ bambooshoots<br>
├─ betel<br>
   ...<br>
├─ waterbamboor<br>


# 壓縮圖片解析度

In [2]:
save_path = './dataset_1200/'

dataset_dir = "./dataset"

#依序處理每個類別資料夾
for i, image_class in enumerate(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, image_class)
    if os.path.isdir(class_path):
        save_class_path = os.path.join(save_path, image_class)
        #如果儲存路徑不存在自動建立
        if not os.path.exists(save_class_path):
            os.makedirs(save_class_path)
        #將影像resize成1200*1200
        for image_id in tqdm(os.listdir(class_path)):
            image = cv2.imread(os.path.join(class_path, image_id))
            image = cv2.resize(image, (1200, 1200))
            cv2.imwrite(os.path.join(save_class_path, image_id), image)

100%|██████████| 3194/3194 [09:13<00:00,  5.77it/s]


# 計算資料集影像RGB通道的標準差和平均值

In [4]:
dataset_dir = "./dataset_1200"
mean_list = []
std_list = []
for i, image_class in enumerate(os.listdir(dataset_dir)):
    if os.path.isdir(os.path.join(dataset_dir, image_class)):
        for image_id in tqdm(os.listdir(os.path.join(dataset_dir, image_class))):
            image = cv2.imread(os.path.join(dataset_dir, image_class, image_id))
            image = image / 255
            mean_r = np.mean(image[:,:,2])
            mean_g = np.mean(image[:,:,1])
            mean_b = np.mean(image[:,:,0])
            std_r = np.std(image[:,:,2])
            std_g = np.std(image[:,:,1])
            std_b = np.std(image[:,:,0])
            mean_list.append([mean_r, mean_g, mean_b])
            std_list.append([std_r, std_g, std_b])

 27%|██▋       | 855/3158 [00:55<02:28, 15.47it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  9%|▉         | 216/2331 [00:14<02:21, 14.99it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
color_dict = {0: 'r', 1: 'g', 2:'b'}

mean_np_array = np.array(mean_list)
std_np_array = np.array(std_list)

for i in range(3):
    print(f'{color_dict[i]} mean: {np.mean(mean_np_array[:,i])} std: {np.std(mean_np_array[:,i])}')

r mean: 0.45880166530111677 std: 0.08075012492584159
g mean: 0.48741071776250683 std: 0.07701888427745267
b mean: 0.42037818746947636 std: 0.09533666046113468


# public和private影像

In [3]:
save_path = './testset_1200/images/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
dataset_dir = "./testset"
for image_id in tqdm(os.listdir(dataset_dir)):
    image = cv2.imread(os.path.join(dataset_dir, image_id))
    image = cv2.resize(image, (1200, 1200))
    cv2.imwrite(os.path.join(save_path, image_id), image)

100%|██████████| 11148/11148 [30:54<00:00,  6.01it/s]


In [ ]:
import os
import cv2
from tqdm import tqdm

save_path = './private_testset_1200/images/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
dataset_dir = "./private_testset"
for image_id in tqdm(os.listdir(dataset_dir)):
    image = cv2.imread(os.path.join(dataset_dir, image_id))
    image = cv2.resize(image, (1200, 1200))
    cv2.imwrite(os.path.join(save_path, image_id), image)

 92%|█████████▏| 10271/11160 [25:40<02:07,  6.98it/s]

# 裁切準心部分

In [ ]:
df = pd.read_csv('./tag_locCoor.csv', encoding='big5')

save_path = './dataset_p4/'

dataset_dir = "./dataset"
for i, image_class in enumerate(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, image_class)
    if os.path.isdir(class_path):
        save_class_path = os.path.join(save_path, image_class)
        if not os.path.exists(save_class_path):
            os.makedirs(save_class_path)
        for image_id in tqdm(os.listdir(class_path)):
            row = df[df['Img'] == image_id]
            image = cv2.imread(os.path.join(class_path, image_id))

            height, width, _ = np.shape(image)

            '''if height + width > 5000:
                length = 512
            else:
                length = 256'''

            length = max(height, width) // 4

            try:
                if width > height:
                    #cv2.circle(image, (int(width/2 + row['target_y']), int(height/2 + row['target_x'])), 5, color=(0,0,255))
                    #print(int(width/2 + row['target_y']), int(height/2 + row['target_x']))
                    x, y = int(width/2 + row['target_y']), int(height/2 + row['target_x'])
                    new_image = image[y-length:y+length,x-length:x+length]
                    
                else:
                    #cv2.circle(image, (int(width/2 + row['target_x']), int(height/2 - row['target_y'])), 5, color=(0,0,255))
                    #print(int(width/2 + row['target_x']), int(height/2 - row['target_y']))
                    x, y = int(width/2 + row['target_x']), int(height/2 - row['target_y'])
                    new_image = image[y-length:y+length,x-length:x+length]

                new_image = cv2.resize(new_image, (448, 448))
                #new_image = cv2.resize(new_image, (224, 224))
                cv2.imwrite(os.path.join(save_class_path, image_id), new_image)
            except:
                x, y = int(width/2), int(height/2)
                new_image = image[y-length:y+length,x-length:x+length]
                new_image = cv2.resize(new_image, (448, 448))
                cv2.imwrite(os.path.join(save_class_path, image_id), new_image)